In [6]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError

In [7]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [8]:
sql = MySQLConnection('oskars_s','ec2-13-53-140-213.eu-north-1.compute.amazonaws.com','dati')

Enter password for user oskars_s········


In [5]:
a_code = pd.read_csv('access-code.csv',';')
a_code_pword_rec = pd.read_csv('access-code-password-recovery-code.csv',sep=';')
un_pw_rec_code = pd.read_csv('username-password-recovery-code.csv',sep=';')
un = pd.read_csv('username.csv',sep=';')
email = pd.read_csv('email.csv',sep=';')
un_email = pd.read_csv('username-or-email.csv',sep=';')
email_pw_rec = pd.read_csv('email-password-recovery-code.csv',sep=';')

In [6]:
a_code.columns = ['Identifier', 'Firstname','Lastname']
a_code_pword_rec.columns = ['Identifier','Accesscode','Recoverycode','Firstname','Lastname','Department','Location']
un_pw_rec_code.columns = ['Username','Identifier','Onetimepassword','Recoverycode','Firstname','Lastname','Department','Location']
un.columns = ['Username', 'Identifier', 'Firstname','Lastname']
email.columns = ['Loginemail','Identifier', 'Firstname','Lastname']
un_email.columns = ['Username','Loginemail','Identifier', 'Firstname','Lastname']
email_pw_rec.columns = ['Loginemail','Identifier','Onetimepassword','Recoverycode', 'Firstname','Lastname','Department','Location']

In [7]:
sql.write_to_db(a_code,'access_code')
sql.write_to_db(a_code_pword_rec,'access_code_password_recovery_code')
sql.write_to_db(un_pw_rec_code,'username_password_recovery_code')
sql.write_to_db(un,'username')
sql.write_to_db(email,'email')
sql.write_to_db(un_email,'username_or_email')
sql.write_to_db(email_pw_rec,'email_password_recovery_code')
check = True